In [14]:
from pathlib import Path
import json
import random
import os

In [23]:
train_frac = 0.8
n_folds = 5
dataroot = Path("/media/smbshare/3Tpioneer_bids")

In [ ]:
with open("/home/srs-9/Projects/prl_project/data/subject-sessions-longit.json", 'r') as f:
    subject_sessions = json.load(f)

with open("/home/srs-9/Projects/prl_project/training/test_train0/subjects-to-train.txt", 'r') as f:
    subjects = [line.strip() for line in f.readlines()]

scan_dirs = []
for subid in subjects:
    sessions = sorted(subject_sessions[subid])
    scan_dir = dataroot / f"sub-ms{subid}" / f"ses-{sessions[0]}"
    scan_dirs.append(scan_dir)

In [19]:
data = []
for path in scan_dirs:
    image = path / "flair.phase.t1.nii.gz"
    if not image.exists():
        print(f"{str(image)} not found")
        continue
    label = path / "lesion.t3m20" / "lesion_and_prl_labels_tmp.nii.gz"
    if not label.exists():
        print(f"{str(label)} not found")
        continue
    data.append({"image": str(image), "label": str(label)})

/media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.phase.t1.nii.gz not found


In [20]:
random.shuffle(data)
train_number = int(len(data) * 0.8) - (int(len(data) * 0.8) % n_folds)
test_data = data.copy()
train_data = [test_data.pop() for _ in range(train_number)]

In [24]:
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

datalist_file = os.path.join(os.getcwd(), "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)